In [1]:
import sys

sys.path.append("./myutils")
from data_generator import (
    DataGeneratorParam,
    TrainDataParam,
    RealDataParam,
    ExpandedDataParam,
    DataGenerator,
)
from train_rhat import train_rhat
from r0_CI import construct_r0_CIs
from true_models import generate_true_models, generate_bias_models
from learner import build_learner
from utils import get_model_directory_uri
import itertools
import numpy as np
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="You are using `torch.load` with `weights_only=False`",
    category=FutureWarning
)

# All parameters

In [3]:
Q = 100  # Z in R^Q
P = 100  # X in R^P
# Y in R
OUTPUT_DIRECTORY_URI = "./results"

In [4]:
r0, g0, f0 = generate_true_models(Q, P)
DATA_GENERATOR_PARAM = DataGeneratorParam(p=P, q=Q, r0=r0, g0=g0, f0=f0)

TRAIN_DATA_PARAM = TrainDataParam(n_train=50000)

In [5]:
r0_LEARNERS = {
    # "linear": build_learner(model_type='ols'),
    "kernel": build_learner(model_type="krr"),
    "xgboost": build_learner(model_type="xgb", output_dim=P),
    "neural_net_128x128_1500_64": build_learner(
        model_type="mlp",
        input_dim=P,
        output_dim=Q,
        hidden_layers=[128, 128],
        epochs=1500,
        batch_size=64,
    ),
    "random_forest": build_learner(
        model_type="rf",
        output_dim=P,
        max_features="sqrt",
    ),
    # "random_forest_20dep_100ests": build_learner(
    #     model_type="rf",
    #     output_dim=P,
    #     max_depth=20,
    #     n_estimators=100,
    #     max_features="sqrt",
    # ),
    # "random_forest_40dep_100ests": build_learner(
    #     model_type="rf",
    #     output_dim=P,
    #     max_depth=40,
    #     n_estimators=100,
    #     max_features="sqrt",
    # ),
}

In [6]:
nn_bias_1, nn_bias_2 = generate_bias_models(Q, P)
BIAS_FUNCS = [nn_bias_1, nn_bias_2]
BIAS_SCALES = [0, 1, 10]
N_REALS = [100, 1000]
R_EXPANDEDS = [0.001, 0.01, 0.1]

REAL_DATA_PARAMS = [
    RealDataParam(bias_func=bias_func, bias_scale=bias_scale, n_real=n_real)
    for bias_func, bias_scale, n_real in itertools.product(
        BIAS_FUNCS, BIAS_SCALES, N_REALS
    )
]
print("Number of real data param combos:", len(REAL_DATA_PARAMS))

EXPANDED_DATA_PARAMS = [
    ExpandedDataParam(n_expanded=10 * n_real, r_expanded=r)
    for n_real, r in itertools.product(N_REALS, R_EXPANDEDS)
]
print("Number of expanded data param combos:", len(EXPANDED_DATA_PARAMS))

Number of real data param combos: 12
Number of expanded data param combos: 6


In [7]:
combinations = list(
    itertools.product(r0_LEARNERS.items(), REAL_DATA_PARAMS, EXPANDED_DATA_PARAMS)
)
print(f"There are {len(combinations)} combinations to run.")

There are 288 combinations to run.


# Retrieve all trained models

In [8]:
def train_single_model(args):
    learner_name, learner = args
    model_directory_uri = get_model_directory_uri(
        data_generator_param=DATA_GENERATOR_PARAM,
        train_data_param=TRAIN_DATA_PARAM,
        r0_learner_name=learner_name,
        output_directory_uri=OUTPUT_DIRECTORY_URI,
    )
    rhat = train_rhat(
        data_generator_param=DATA_GENERATOR_PARAM,
        train_data_param=TRAIN_DATA_PARAM,
        model_directory_uri=model_directory_uri,
        learner_name=learner_name,
        learner=learner,
        fresh=False,
    )
    return model_directory_uri, rhat

In [9]:
trained_models = {}

with ProcessPoolExecutor() as executor:
    futures = {
        executor.submit(train_single_model, item): item[0]  # learner_name
        for item in r0_LEARNERS.items()
    }
    for future in tqdm(as_completed(futures), total=len(futures), desc="Training Models"):
        learner_name = futures[future]
        model_directory_uri, rhat = future.result()
        trained_models[learner_name] = (model_directory_uri, rhat)

Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

Reading rhat...
Reading rhat...


Reading rhat...

Reading rhat...

kernel training MSE = 6039.233176466542
'train_rhat' executed in 0.581983s


Training Models:  25%|██▌       | 1/4 [00:00<00:01,  1.72it/s]

xgboost training MSE = 2670.9454665857
'train_rhat' executed in 1.510732s


Training Models:  50%|█████     | 2/4 [00:02<00:02,  1.40s/it]

neural_net_128x128_1500_64 training MSE = 1.1814813298133593
'train_rhat' executed in 6.596700s


Training Models:  75%|███████▌  | 3/4 [00:07<00:03,  3.21s/it]

random_forest training MSE = 1533.9805656345966
'train_rhat' executed in 55.416403s


Training Models: 100%|██████████| 4/4 [03:17<00:00, 49.32s/it]


# Construct r0_CIs


In [10]:
def run_single_experiment(
    r0_learner_name,
    model_directory_uri,
    rhat,
    data_generator_param,
    real_data_param,
    expanded_data_param,
    r0,
    fresh=False,
):
    r0_CIs, coverage, avg_me = construct_r0_CIs(
        data_generator_param=data_generator_param,
        real_data_param=real_data_param,
        expanded_data_param=expanded_data_param,
        model_directory_uri=model_directory_uri,
        rhat=rhat,
        r0=r0,
        fresh=fresh,
    )

    return {
        "r0_learner_name": r0_learner_name,
        "bias_func": real_data_param.bias_func.__class__.__name__,
        "bias_scale": real_data_param.bias_scale,
        "n_real": real_data_param.n_real,
        "n_expanded": expanded_data_param.n_expanded,
        "r_expanded": expanded_data_param.r_expanded,
        "coverage": coverage,
        "avg_me": avg_me,
    }

In [11]:
def run_combination(args):
    (r0_learner_name, _), real_data_param, expanded_data_param = args
    model_directory_uri, rhat = trained_models[r0_learner_name]
    
    # print(f"Running {r0_learner_name}, bias={real_data_param.bias_scale}, r={expanded_data_param.r_expanded}")
    return run_single_experiment(
        r0_learner_name=r0_learner_name,
        model_directory_uri=model_directory_uri,
        rhat=rhat,
        data_generator_param=DATA_GENERATOR_PARAM,
        real_data_param=real_data_param,
        expanded_data_param=expanded_data_param,
        r0=r0,
        fresh=False,
    )

In [12]:
results = []
with ThreadPool() as pool:
    for result in tqdm(pool.imap_unordered(run_combination, combinations, chunksize=1), total=len(combinations)):
        results.append(result)

  0%|          | 0/288 [00:00<?, ?it/s]

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 CIs...

Constructing r0 

  0%|          | 1/288 [08:49<42:14:44, 529.91s/it]

Coverage: 0.389
Average ME: 26.508
'construct_r0_CIs' executed in 529.887754s
Constructing r0 CIs...

Saving...
Saving...
Saving...


  1%|          | 2/288 [08:51<17:24:47, 219.19s/it]

...

Saving...
Coverage: 0.040
Average ME: 71.296
'construct_r0_CIs' executed in 43234.555031s


100%|█████████▉| 287/288 [93:33:05<30:48, 1848.86s/it]  

Saving...
Coverage: 0.040
Average ME: 71.296
'construct_r0_CIs' executed in 43179.952417s


100%|██████████| 288/288 [93:33:18<00:00, 1169.44s/it]

Saving...
Coverage: 0.039
Average ME: 71.296
'construct_r0_CIs' executed in 38988.081266s


In [13]:
results = pd.DataFrame(results)
print(results)
results.to_csv("results.csv", index=False)

    r0_learner_name bias_func  bias_scale  n_real  n_expanded  r_expanded  \
0            kernel   NNBias1           0     100        1000       0.010   
1            kernel   NNBias1           0     100        1000       0.001   
2            kernel   NNBias1           0     100        1000       0.100   
3            kernel   NNBias1           1     100        1000       0.001   
4            kernel   NNBias1           1     100        1000       0.010   
..              ...       ...         ...     ...         ...         ...   
283   random_forest   NNBias2          10     100       10000       0.010   
284   random_forest   NNBias2          10     100       10000       0.100   
285   random_forest   NNBias2          10    1000       10000       0.001   
286   random_forest   NNBias2          10    1000       10000       0.010   
287   random_forest   NNBias2          10    1000       10000       0.100   

     coverage      avg_me  
0     0.38862   26.507606  
1     0.38862   26.

In [ ]:
from IPython.display import Audio, display

# Replace 'sound.mp3' with the path to your MP3 file.
display(Audio("/u/home/y/yqg36/alarm.mp3", autoplay=True))